In [1]:
import re
from datetime import datetime, timedelta
import pandas as pd
from config import *

In [2]:
today = datetime.today() + timedelta(days=1)
date_rng = pd.date_range(start=START_DATE, end=END_DATE, freq='D')

# 데이터프레임 생성
설정맵 = pd.DataFrame(columns=MEMBER, index=date_rng)
벌금맵 = 설정맵
설정맵 = 설정맵.fillna("설정X")
벌금맵 = 벌금맵.fillna(0)

def make_off_every_sunday():
    for idx, _ in 설정맵.iterrows():
        if idx.weekday() == 6:
            설정맵.loc[idx] = 'off'

def avoid_over_date(date_str, 설정맵, 벌금맵):
    if (date_str not in 설정맵.index):
        설정맵.loc[date_str] = '설정X'
        벌금맵.loc[date_str] = 0

make_off_every_sunday()

In [3]:
file = pd.read_csv("7기.csv", encoding='UTF8')

filter_option = f"{pattern_set}|{pattern_success}|{pattern_fail}|{pattern_off}|{pattern_offs}|{pattern_retry_set}|{pattern_retry_sucess}"
filtered_rows = file[file['Message'].str.contains(filter_option, regex=True)]
filtered_rows = filtered_rows.reset_index(drop=True, inplace=False)
filtered_rows['Late'] = 'O'

len_of_rows = filtered_rows.shape[0]
# filtered_rows

In [4]:
def set_deadline(USER, line):
    if USER in ['권세한', '임혁규', '이신후', '이준수']:
        DEADLINE = 21
    elif 3 <= datetime.strptime(line, "%Y%m%d").month <= 6:
        DEADLINE = 11    
    else :
        DEADLINE = 10
        
    return DEADLINE
        

In [5]:
# 지각 여부 확인
i = 0
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]
    LATE = filtered_rows['Late'][index]

    result_set = str(re.findall(pattern_set, MESSAGE))[2:][:-2]
    result_retry_set = str(re.findall(pattern_retry_set, MESSAGE))[2:][:-2]

    result_success = str(re.findall(pattern_success, MESSAGE))[2:][:-2]
    result_retry_success = str(re.findall(pattern_retry_sucess, MESSAGE))[2:][:-2]
    result_fail = str(re.findall(pattern_fail, MESSAGE))[2:][:-2]
        
    time_str = DATE
    time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')
    
    if len(result_success) >= 1 | len(result_fail) >= 1 | len(result_retry_success) >= 1:
        if  len(result_retry_success) > 1:
            date_str_next = '2024' + result_retry_success[:4]
            
        elif len(result_fail) > 1:
            date_str_next = '2024' + result_fail[:4]
        
        else:
            date_str_next = '2024' + result_success[:4]

        DEADLINE = set_deadline(USER, date_str_next)
            
        date_obj_next = datetime.strptime(date_str_next + f'{DEADLINE}', "%Y%m%d%H") + timedelta(days=1)
        formatted_date_next = date_obj_next.strftime("%Y-%m-%d %H:%M:%S")
        time_difference_next = time_obj - datetime.strptime(formatted_date_next, "%Y-%m-%d %H:%M:%S")    
        date_str_next = date_obj_next.strftime("%Y-%m-%d")
        date_index_next = datetime.strptime(date_str_next, '%Y-%m-%d')
    
        if time_difference_next.days > 0 or (time_difference_next.days == 0 and time_difference_next.seconds >= 60):
            filtered_rows['Late'][index] = '지각'
        
    
    elif len(result_set) >= 1 | len(result_retry_set) >= 1:
        if len(result_retry_set) > 1:
            date_str = '2024' + result_retry_set[:4]
        else:
            date_str = '2024' + result_set[:4]
            
        DEADLINE = set_deadline(USER, date_str)

        date_obj = datetime.strptime(date_str + f'{DEADLINE}', "%Y%m%d%H")
        formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")
        time_difference = time_obj - datetime.strptime(formatted_date, "%Y-%m-%d %H:%M:%S")
        date_str = date_obj.strftime("%Y-%m-%d")
        date_index = datetime.strptime(date_str, '%Y-%m-%d')
    
        if time_difference.days > 0 or (time_difference.days == 0 and time_difference.seconds >= 60):
            filtered_rows['Late'][index] = '지각'

In [6]:
#설정 코드
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]
    LATE = filtered_rows['Late'][index]

    result_set = str(re.findall(pattern_set, MESSAGE))[2:][:-2]
    result_retry_set = str(re.findall(pattern_retry_set, MESSAGE))[2:][:-2]

    if len(result_set) <= 1 | len(result_retry_set) <= 1:
        continue
    elif len(result_retry_set) > 1:
        date_str = '2024' + result_retry_set[:4]
    else:
        date_str = '2024' + result_set[:4]

    DEADLINE = set_deadline(USER, date_str)

    date_obj = datetime.strptime(date_str + f'{DEADLINE}', "%Y%m%d%H")

    time_str = DATE
    time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')

    formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")

    time_difference = time_obj - datetime.strptime(formatted_date, "%Y-%m-%d %H:%M:%S")

    # 결과 확인
    date_str = date_obj.strftime("%Y-%m-%d")

    date_index = datetime.strptime(date_str, '%Y-%m-%d')
    
    # 설정은 최초 1회만 가능 (재도전을 위해 off 추가)
    if 설정맵.loc[date_index, USER] in ['설정X', 'off']:
        if filtered_rows['Late'][index] == '지각':
            설정맵.loc[date_index, USER] = '지각'
        else: 
            설정맵.loc[date_index, USER] = 'O'

In [7]:
# 성공 코드
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]
    LATE = filtered_rows['Late'][index]

    result_success = str(re.findall(pattern_success, MESSAGE))[2:][:-2]
    result_retry_success = str(re.findall(pattern_retry_sucess, MESSAGE))[2:][:-2]
    result_fail = str(re.findall(pattern_fail, MESSAGE))[2:][:-2]

    if len(result_success) <= 1 | len(result_fail) <= 1 | len(result_retry_success) <= 1:
        continue
    
    if  len(result_retry_success) > 1:
        re.search(pattern_retry_sucess, result_retry_success)
        date_range_str = re.search(r'\d{4}\s?\w+\s?\d{4}', result_retry_success).group()
        date_str = datetime.strptime('2024'+date_range_str[:4], "%Y%m%d")
        target_date = datetime.strptime('2024'+date_range_str[-4:], "%Y%m%d")
        
        if 설정맵.loc[date_str, USER] in ['설정X', 'O', '지각', 'off']:

            if 설정맵.loc[date_str, USER] == 'O':
                설정맵.loc[date_str, USER] = '성공'
                설정맵.loc[target_date, USER] = '재성공'
                continue

            elif 설정맵.loc[date_str, USER] == '지각':
                설정맵.loc[date_str, USER] = '성공'
                설정맵.loc[target_date, USER] = '지각+재성공'
                continue
        
    elif len(result_fail) >= 1:
        date_str = '2024' + result_fail[:4]
    
    else:
        date_str = '2024' + result_success[:4]
        
    date_obj = datetime.strptime(date_str, "%Y%m%d")

    date_str = date_obj.strftime("%Y-%m-%d")
    
    avoid_over_date(date_str, 설정맵, 벌금맵)

    if 설정맵.loc[date_str, USER] in ['설정X', 'O', '지각']:
        if filtered_rows['Late'][index]=='지각':
            if len(result_success)  > 1:
                설정맵.loc[date_str, USER] = '지각+성공'
                continue
            
            elif len(result_fail)  > 1: 
                설정맵.loc[date_str, USER] = '지각+실패'
                continue
        
        elif 설정맵.loc[date_str, USER] == 'O':
            if len(result_success)  > 1:
                설정맵.loc[date_str, USER] = '성공'
                continue
            
            elif len(result_fail)  > 1: 
                설정맵.loc[date_str, USER] = '실패'
                continue
            
        elif 설정맵.loc[date_str, USER] == '지각':
            if len(result_success)  > 1:
                설정맵.loc[date_str, USER] = '지각+성공'
                continue
            
            elif len(result_fail)  > 1: 
                설정맵.loc[date_str, USER] = '지각+실패'
                continue

In [8]:
# off 코드

for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]

    result_off = str(re.findall(pattern_off, MESSAGE))[2:][:-2]
    result_offs = str(re.findall(pattern_offs, MESSAGE))[2:][:-2]

    if len(result_off) <= 1:
        continue
    else:
        date_str = '2024' + result_off[:4]
    
    date_obj = datetime.strptime(date_str, "%Y%m%d")
    date_str = date_obj.strftime("%Y-%m-%d")
    
    if (result_off[-3:] == 'off') and ((설정맵.loc[date_str, USER] == 'O') or (설정맵.loc[date_str, USER] == '설정X')):
        설정맵.loc[date_str, USER] = 'off'
        
    if re.search(pattern_offs, result_offs):
        date_range_str = re.search(r'\d{4}-\d{4}', result_offs).group()
        start_date = datetime.strptime('2024'+date_range_str[:4], "%Y%m%d")
        end_date = datetime.strptime('2024'+date_range_str[5:9], "%Y%m%d")
        
        설정맵.loc[start_date:end_date, USER] = 'off'

In [9]:
설정맵

,박연수,김건영,권세한,박지은,송호석,이신후,이준수,임혁규
2024-05-15,성공,성공,off,성공,off,off,성공,off
2024-05-16,성공,성공,성공,성공,지각+성공,off,off,off
2024-05-17,실패,off,off,실패,지각+성공,off,지각+성공,off
2024-05-18,off,off,지각+실패,실패,off,off,off,off
2024-05-19,off,off,off,실패,off,off,지각+실패,off
2024-05-20,성공,off,성공,성공,지각+성공,off,지각+성공,off
2024-05-21,실패,실패,성공,성공,지각+성공,성공,off,성공
2024-05-22,지각+성공,설정X,off,지각+실패,성공,성공,지각+성공,off
2024-05-23,지각+성공,off,실패,지각+성공,off,off,지각+성공,성공
2024-05-24,off,설정X,실패,실패,지각+실패,성공,지각+실패,off


In [10]:
# 5일 연속 성공 여부 확인
def check_consecutive_success(user_data):
    consecutive_count = 0
    for status in user_data:
        if status == '성공':
            consecutive_count += 1
            if consecutive_count == 5:
                return True
        else:
            consecutive_count = 0
    return False

# 부여 까방권
for user in 설정맵.columns:
    user_data = 설정맵[user].dropna()  # NaN 값 제외
    if check_consecutive_success(user_data):
        print(f'{user}님에게 까방권 부여!')

박연수님에게 까방권 부여!
이신후님에게 까방권 부여!
임혁규님에게 까방권 부여!


In [11]:
# 억지로 예외처리 

In [12]:
max_consecutive_failures = 3
max_consecutive_successes = 3
max_consecutive_late_failures = 3
max_consecutive_late_sucesses = 3
# 설정맵을 순회하면서 벌금맵에 적절한 값 설정
for person in 설정맵.columns:
    FAIL_STATE = 3000
    LATE_STATE = 1000

    # 연속으로 나온 실패 및 성공 횟수 초기화
    consecutive_failures = 0
    consecutive_successes = 0
    consecutive_late_successes = 0
    consecutive_late_failures = 0

    for date, status in 설정맵[person].items():
        if pd.notna(status):  # NaN이 아닌 경우에만 처리
            time_str = str(date)
            time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')
            formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")

            time_difference = time_obj - datetime.today()
            
            if '지각' in str(status):  # '지각'이 문자열에 포함되어 있는지 확인
                벌금맵[person][date] += LATE_STATE
                consecutive_late_failures += 1
                consecutive_late_successes = 0

            else : 
                consecutive_late_successes += 1
                consecutive_late_failures = 0
                
            if '성공' in str(status):
                벌금맵[person][date] += 0
                consecutive_failures = 0
                consecutive_successes += 1
                
            elif '실패' in str(status):
                벌금맵[person][date] += FAIL_STATE
                consecutive_failures += 1
                consecutive_successes = 0
                
            elif status == 'off':
                벌금맵[person][date] = 0

            elif status == '재성공':
                벌금맵[person][date] = 0
                consecutive_failures = 0
                consecutive_successes += 1

            elif status == '설정X' and  time_difference.days < -2:
                벌금맵[person][date] = FAIL_STATE+LATE_STATE
                consecutive_late_failures += 1
                consecutive_failures += 1
                consecutive_successes = 0
    
            elif status == 'O' and  time_difference.days < -2 :    
                벌금맵[person][date] = FAIL_STATE+LATE_STATE
                consecutive_late_failures += 1
                consecutive_failures += 1
                consecutive_successes = 0
    
            # 연속으로 나온 실패 및 성공 횟수가 설정값 이상이면 벌금 증가 또는 감소
            if consecutive_failures >= max_consecutive_failures:
                FAIL_STATE = 5000
                
            elif consecutive_successes >= max_consecutive_successes:
                FAIL_STATE = 3000

            if consecutive_late_failures >= max_consecutive_late_failures:
                LATE_STATE = 2000
                
            elif consecutive_late_successes >= max_consecutive_late_sucesses:
                LATE_STATE = 1000

In [13]:
벌금맵

,박연수,김건영,권세한,박지은,송호석,이신후,이준수,임혁규
2024-05-15,0,0,0,0,0,0,0,0
2024-05-16,0,0,0,0,1000,0,0,0
2024-05-17,3000,0,0,3000,1000,0,1000,0
2024-05-18,0,0,4000,3000,0,0,0,0
2024-05-19,0,0,0,3000,0,0,4000,0
2024-05-20,0,0,0,0,1000,0,1000,0
2024-05-21,3000,3000,0,0,1000,0,0,0
2024-05-22,1000,4000,0,6000,0,0,1000,0
2024-05-23,1000,0,3000,1000,0,0,1000,0
2024-05-24,0,4000,3000,5000,4000,0,4000,0


In [14]:
# datetime 인덱스를 'YYYY-MM-DD' 포맷으로 변경
벌금맵.index = 벌금맵.index.map(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x)
설정맵.index = 설정맵.index.map(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x)

벌금맵.loc['합계'] = 벌금맵.sum()

# 각 열의 합을 계산하여 마지막 행에 추가

벌금맵.loc['입금'] = 박연수_벌금, 김건영_벌금, 권세한_벌금, 박지은_벌금, 송호석_벌금, 이신후_벌금, 이준수_벌금, 임혁규_벌금
벌금맵.loc['벌금 연체료'] = 0, 0, 0, 0, 0, 0, 0 ,0
벌금맵.loc['미납'] = 벌금맵.loc['합계'] + 벌금맵.loc['벌금 연체료'] - 벌금맵.loc['입금']
벌금맵.loc['지원금'] = 0
벌금맵.loc[''] = ' '

# 총 벌금
total_penalty = 벌금맵.loc['합계'].sum()
deposit_penalty = 벌금맵.loc['입금'].sum()

# off 제외 설정맵의 개수
total_cells_excluding_off = 설정맵.apply(lambda x: x != 'off').sum().sum()

for col in 설정맵.columns:
    col_off_count = 설정맵[col].apply(lambda x: x != 'off').sum()
    subsidy_proportion = col_off_count / total_cells_excluding_off
    subsidy =int(round(subsidy_proportion * total_penalty, -2))
    벌금맵[col]['지원금'] = subsidy

total_subsidy = 벌금맵.loc['지원금'].sum()

# print("총 벌금 :" , total_penalty)
# print("총 지원금 :" , total_subsidy)
# print("총 입금 :" , deposit_penalty)

# 출력
벌금맵

,박연수,김건영,권세한,박지은,송호석,이신후,이준수,임혁규
2024-05-15,0,0,0,0,0,0,0,0
2024-05-16,0,0,0,0,1000,0,0,0
2024-05-17,3000,0,0,3000,1000,0,1000,0
2024-05-18,0,0,4000,3000,0,0,0,0
2024-05-19,0,0,0,3000,0,0,4000,0
2024-05-20,0,0,0,0,1000,0,1000,0
2024-05-21,3000,3000,0,0,1000,0,0,0
2024-05-22,1000,4000,0,6000,0,0,1000,0
2024-05-23,1000,0,3000,1000,0,0,1000,0
2024-05-24,0,4000,3000,5000,4000,0,4000,0


In [15]:
벌금맵.loc['입금'].sum()

0

In [16]:
설정맵

,박연수,김건영,권세한,박지은,송호석,이신후,이준수,임혁규
2024-05-15,성공,성공,off,성공,off,off,성공,off
2024-05-16,성공,성공,성공,성공,지각+성공,off,off,off
2024-05-17,실패,off,off,실패,지각+성공,off,지각+성공,off
2024-05-18,off,off,지각+실패,실패,off,off,off,off
2024-05-19,off,off,off,실패,off,off,지각+실패,off
2024-05-20,성공,off,성공,성공,지각+성공,off,지각+성공,off
2024-05-21,실패,실패,성공,성공,지각+성공,성공,off,성공
2024-05-22,지각+성공,설정X,off,지각+실패,성공,성공,지각+성공,off
2024-05-23,지각+성공,off,실패,지각+성공,off,off,지각+성공,성공
2024-05-24,off,설정X,실패,실패,지각+실패,성공,지각+실패,off


In [17]:
맵 = pd.concat((벌금맵, 설정맵))

맵.to_csv('결과.csv')

In [18]:
print('마지막 지원 확인일 : 0415')

마지막 지원 확인일 : 0415
